In [8]:
## import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

nltk.download('words')
words = set(nltk.corpus.words.words())

[nltk_data] Error loading stopwords: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>
[nltk_data] Error loading words: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>


In [9]:
import pandas as pd
import nltk 
import numpy as np
import string
from textblob import TextBlob
from collections import Counter
from matplotlib import pyplot as plt
import io
import unicodedata
import re
import string
from fuzzywuzzy import fuzz  
import difflib
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from wordcloud import WordCloud

In [10]:
df = pd.read_csv('train.csv')
df.head()

,tweet,label
0,Well said sonu..you have courage to stand agai...,OAG
1,"Most of Private Banks ATM's Like HDFC, ICICI e...",NAG
2,"Now question is, Pakistan will adhere to this?",OAG
3,Pakistan is comprised of fake muslims who does...,OAG
4,"??we r against cow slaughter,so of course it w...",NAG


In [11]:
#Data Cleaning

In [12]:
from nltk.stem.wordnet import WordNetLemmatizer

In [13]:
from nltk.corpus import stopwords
lemmatizer = WordNetLemmatizer()
stop_words = stopwords.words('english')
VERB_CODES = {'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'}

In [14]:
## new
def preprocess_sentences(text):
    text = text.lower()
    text=text.strip()  #get rid of leading/trailing whitespace 
      
    temp_sent =[]
    words = nltk.word_tokenize(text)
    tags = nltk.pos_tag(words)
    for i, word in enumerate(words):
        if tags[i][1] in VERB_CODES:
            lemmatized = lemmatizer.lemmatize(word, 'v')
        else:
            lemmatized = lemmatizer.lemmatize(word)
        if lemmatized not in stop_words and lemmatized.isalpha() and len(lemmatized)>2:
            temp_sent.append(lemmatized)

    finalsent = ' '.join(temp_sent)
    
    return finalsent


df["Tweet_clean"]= df["tweet"].apply(preprocess_sentences)

In [15]:
df

,tweet,label,Tweet_clean
0,Well said sonu..you have courage to stand agai...,OAG,well say sonu courage stand dadagiri muslim
1,"Most of Private Banks ATM's Like HDFC, ICICI e...",NAG,private bank atm like hdfc icici etc cash publ...
2,"Now question is, Pakistan will adhere to this?",OAG,question pakistan adhere
3,Pakistan is comprised of fake muslims who does...,OAG,pakistan comprise fake muslim know meaning uni...
4,"??we r against cow slaughter,so of course it w...",NAG,cow slaughter course stop leather manufacturin...
...,...,...,...
11994,They belong to you flight dirty terrorist coun...,OAG,belong flight dirty terrorist country india
11995,"Really motivating programme, congratulations t...",NAG,really motivate programme congratulation cnbc ...
11996,fabricated news,OAG,fabricate news
11997,What's wrong with you secular idiots,OAG,wrong secular idiot


In [16]:
df['Tweet_clean'].isnull().sum()

0

In [17]:
df['Tweet_clean']

0              well say sonu courage stand dadagiri muslim
1        private bank atm like hdfc icici etc cash publ...
2                                 question pakistan adhere
3        pakistan comprise fake muslim know meaning uni...
4        cow slaughter course stop leather manufacturin...
                               ...                        
11994          belong flight dirty terrorist country india
11995    really motivate programme congratulation cnbc ...
11996                                       fabricate news
11997                                  wrong secular idiot
11998    look like inevitable political hard ball dialo...
Name: Tweet_clean, Length: 11999, dtype: object

In [18]:
import os
import pandas as pd
import numpy as np
from scipy.stats import randint
import seaborn as sns # used for plot interactive graph.
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics

In [19]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, ngram_range=(1, 2),stop_words='english')
# We transform each complaint into a vector
features = tfidf.fit_transform(df.Tweet_clean).toarray()
labels = df.label
print("Each of the %d tweets is represented by %d features (TF-IDF score of unigrams and bigrams)" %(features.shape))

Each of the 11999 tweets is represented by 5215 features (TF-IDF score of unigrams and bigrams)


In [20]:
features[5217].shape

(5215,)

In [21]:
labels

0        OAG
1        NAG
2        OAG
3        OAG
4        NAG
        ... 
11994    OAG
11995    NAG
11996    OAG
11997    OAG
11998    NAG
Name: label, Length: 11999, dtype: object

In [22]:
# Trying Different Models

In [23]:
from sklearn.metrics import accuracy_score

X = df['Tweet_clean'] # Collection of documents
y = df['label'] # Target or the labels we want to predict
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.15, random_state = 0)

models = [
    RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0),
    LinearSVC(),
    MultinomialNB(),
    LogisticRegression(random_state=0),
]

for model in models:
    model_name = model.__class__.__name__
    
    # Train the model on the training data
    model.fit(X_train, y_train)
    
    # Predict the training labels and calculate the accuracy
    y_train_pred = model.predict(X_train)
    train_acc = accuracy_score(y_train, y_train_pred)
    
    # Predict the test labels and calculate the accuracy
    y_test_pred = model.predict(X_test)
    test_acc = accuracy_score(y_test, y_test_pred)
    
    # Print the train and test accuracy for the model
    print(f"{model_name} train accuracy: {train_acc:.4f}")
    print(f"{model_name} test accuracy: {test_acc:.4f}")


RandomForestClassifier train accuracy: 0.4286
RandomForestClassifier test accuracy: 0.4261
LinearSVC train accuracy: 0.8303
LinearSVC test accuracy: 0.5506
MultinomialNB train accuracy: 0.7062
MultinomialNB test accuracy: 0.5783
LogisticRegression train accuracy: 0.7513
LogisticRegression test accuracy: 0.5867


C:\Users\Akshada Rane\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [36]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# ... (your previous code)

for model in models:
    model_name = model.__class__.__name__
    
    # Train the model on the training data
    model.fit(X_train, y_train)
    
    # Predict the training labels and calculate the accuracy, F1 score, precision, and recall
    y_train_pred = model.predict(X_train)
    train_acc = accuracy_score(y_train, y_train_pred)
    train_f1 = f1_score(y_train, y_train_pred, average = 'weighted')
    train_precision = precision_score(y_train, y_train_pred, average='weighted')
    train_recall = recall_score(y_train, y_train_pred, average='weighted')
    
    # Predict the test labels and calculate the accuracy, F1 score, precision, and recall
    y_test_pred = model.predict(X_test)
    test_acc = accuracy_score(y_test, y_test_pred)
    test_f1 = f1_score(y_test, y_test_pred, average='weighted')
    test_precision = precision_score(y_test, y_test_pred, average='weighted')
    test_recall = recall_score(y_test, y_test_pred, average='weighted')
    
    # Print the evaluation metrics for the model
    print(f"{model_name} train accuracy: {train_acc:.4f} | F1: {train_f1:.4f} | Precision: {train_precision:.4f} | Recall: {train_recall:.4f}")
    print(f"{model_name} test accuracy: {test_acc:.4f} | F1: {test_f1:.4f} | Precision: {test_precision:.4f} | Recall: {test_recall:.4f}")

C:\Users\Akshada Rane\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


RandomForestClassifier train accuracy: 0.4286 | F1: 0.2666 | Precision: 0.5822 | Recall: 0.4286
RandomForestClassifier test accuracy: 0.4261 | F1: 0.2621 | Precision: 0.3265 | Recall: 0.4261
LinearSVC train accuracy: 0.8303 | F1: 0.8295 | Precision: 0.8302 | Recall: 0.8303
LinearSVC test accuracy: 0.5506 | F1: 0.5475 | Precision: 0.5459 | Recall: 0.5506
MultinomialNB train accuracy: 0.7062 | F1: 0.6977 | Precision: 0.7192 | Recall: 0.7062
MultinomialNB test accuracy: 0.5783 | F1: 0.5668 | Precision: 0.5826 | Recall: 0.5783


C:\Users\Akshada Rane\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression train accuracy: 0.7513 | F1: 0.7477 | Precision: 0.7536 | Recall: 0.7513
LogisticRegression test accuracy: 0.5867 | F1: 0.5786 | Precision: 0.5797 | Recall: 0.5867


In [38]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# ... (your previous code)

for model in models:
    model_name = model.__class__.__name__
    
    # Train the model on the training data
    model.fit(X_train, y_train)
    
    # Predict the training labels and calculate the accuracy, F1 score, precision, and recall
    y_train_pred = model.predict(X_train)
    train_acc = accuracy_score(y_train, y_train_pred)
    train_f1 = f1_score(y_train, y_train_pred, average=None)  # Corrected 'none' to 'None'
    train_precision = precision_score(y_train, y_train_pred, average=None)
    train_recall = recall_score(y_train, y_train_pred, average=None)
    
    # Predict the test labels and calculate the accuracy, F1 score, precision, and recall
    y_test_pred = model.predict(X_test)
    test_acc = accuracy_score(y_test, y_test_pred)
    test_f1 = f1_score(y_test, y_test_pred, average=None)
    test_precision = precision_score(y_test, y_test_pred, average=None)
    test_recall = recall_score(y_test, y_test_pred, average=None)
    
    # Print the evaluation metrics for the model
    print(f"{model_name} train accuracy: {train_acc:.4f} | F1: {train_f1} | Precision: {train_precision} | Recall: {train_recall}")
    print(f"{model_name} test accuracy: {test_acc:.4f} | F1: {test_f1} | Precision: {test_precision} | Recall: {test_recall}")


C:\Users\Akshada Rane\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


RandomForestClassifier train accuracy: 0.4286 | F1: [0.03893333 0.59875898 0.00347072] | Precision: [0.5       0.4273062 1.       ] | Recall: [0.02025527 1.         0.00173837]
RandomForestClassifier test accuracy: 0.4261 | F1: [0.03030303 0.59771023 0.        ] | Precision: [0.41666667 0.42623874 0.        ] | Recall: [0.01572327 1.         0.        ]
LinearSVC train accuracy: 0.8303 | F1: [0.80089862 0.8638773  0.81016103] | Precision: [0.81068789 0.8375246  0.84724858] | Recall: [0.79134295 0.89194224 0.77618427]
LinearSVC test accuracy: 0.5506 | F1: [0.47665848 0.64724509 0.4725    ] | Precision: [0.4974359  0.6216545  0.48091603] | Recall: [0.45754717 0.67503303 0.46437346]
MultinomialNB train accuracy: 0.7062 | F1: [0.67931886 0.78128068 0.57037875] | Precision: [0.63903155 0.73486559 0.8156831 ] | Recall: [0.72502775 0.83395435 0.438505  ]
MultinomialNB test accuracy: 0.5783 | F1: [0.53558327 0.67773678 0.4091653 ] | Precision: [0.50206327 0.63406214 0.6127451 ] | Recall: [0.57

C:\Users\Akshada Rane\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression train accuracy: 0.7513 | F1: [0.71995605 0.80872998 0.67716929] | Precision: [0.71280936 0.76430348 0.79740872] | Recall: [0.7272475  0.85863996 0.58843981]
LogisticRegression test accuracy: 0.5867 | F1: [0.52118305 0.69015796 0.46088193] | Precision: [0.5300813  0.63892013 0.5472973 ] | Recall: [0.51257862 0.75033025 0.3980344 ]


In [17]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

X = df['Tweet_clean'] # Collection of documents
y = df['label'] # Target or the labels we want to predict
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=0)

nb = MultinomialNB()
nb.fit(X_train, y_train)

y_train_pred = nb.predict(X_train)
y_test_pred = nb.predict(X_test)

train_acc = accuracy_score(y_train, y_train_pred)
test_acc = accuracy_score(y_test, y_test_pred)

print(f"Multinomial Naive Bayes train accuracy: {train_acc:.4f}")
print(f"Multinomial Naive Bayes test accuracy: {test_acc:.4f}")


Multinomial Naive Bayes train accuracy: 0.7113
Multinomial Naive Bayes test accuracy: 0.5729


# The below is the best model working with almost similar training and testing accuracy

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from imblearn.over_sampling import RandomOverSampler
from sklearn.feature_selection import SelectKBest

In [19]:

X = df['Tweet_clean'] # Collection of documents
y = df['label'] # Target or the labels we want to predict

In [20]:

# Vectorize the text data using TfidfVectorizer
vectorizer = TfidfVectorizer()
X_vec = vectorizer.fit_transform(X)


In [21]:
# Balance the data by oversampling the minority class
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(X_vec, y)

# Perform feature selection using chi-squared test
selector = SelectKBest(chi2, k=1000)
X_resampled_selected = selector.fit_transform(X_resampled, y_resampled)

In [22]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled_selected, y_resampled, test_size=0.2, random_state=42)

In [23]:
# Perform feature selection using chi-squared test
selector = SelectKBest(chi2, k=1000)
X_resampled_selected = selector.fit_transform(X_resampled, y_resampled)

In [24]:
# Define the parameter grid to search over
param_grid = {'alpha': [0.001, 0.01, 0.1, 1.0, 10.0],
              'fit_prior': [True, False]}


In [25]:
# Use cross-validation to evaluate the model's performance on multiple folds of the data and tune the hyperparameters
nb = MultinomialNB()
grid_search = GridSearchCV(nb, param_grid, cv=10)
grid_search.fit(X_resampled_selected, y_resampled)
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation score:", grid_search.best_score_)

Best parameters: {'alpha': 0.01, 'fit_prior': False}
Best cross-validation score: 0.6218622917700742


In [26]:

# Train a final model with the best hyperparameters found by the grid search
nb = MultinomialNB(alpha=grid_search.best_params_['alpha'], fit_prior=grid_search.best_params_['fit_prior'])
nb.fit(X_resampled_selected, y_resampled)





MultinomialNB(alpha=0.01, fit_prior=False)

# Testing & Evaluating model

In [27]:

# Predict the labels for the training and test sets using cross-validation
scores_train = cross_val_score(nb, X_resampled_selected, y_resampled, cv=10)
print("Cross-validation train accuracy:", scores_train.mean())

Cross-validation train accuracy: 0.6218622917700742


In [28]:
# Calculate the accuracy of the classifier on the test set
y_test_pred = nb.predict(X_test)
test_acc = accuracy_score(y_test, y_test_pred)
print(f"Multinomial Naive Bayes test accuracy: {test_acc:.4f}")

Multinomial Naive Bayes test accuracy: 0.6226


In [29]:
from sklearn.metrics import confusion_matrix

# Predict the labels for the test set
y_test_pred = nb.predict(X_test)

# Create a confusion matrix for the test set
conf_matrix = confusion_matrix(y_test, y_test_pred)

# Calculate precision and recall from the confusion matrix
precision = conf_matrix[1,1] / (conf_matrix[1,1] + conf_matrix[0,1])
recall = conf_matrix[1,1] / (conf_matrix[1,1] + conf_matrix[1,0])

# Print the confusion matrix, precision, and recall
print("Confusion matrix:")
print(conf_matrix)
print("Precision:", round(precision,2))
print("Recall:", round(recall,2))


Confusion matrix:
[[643 135 198]
 [355 571 141]
 [254  61 673]]
Precision: 0.81
Recall: 0.62


# Prediction for Adhani Hidenburg Case

In [38]:
import pandas as pd

# Load the Excel file with the new data
new_data = pd.read_excel("Adhani_clean.xlsx")

# Replace missing values with empty strings
new_data['Tweet'] = new_data['Tweet'].fillna('')


# Preprocess the new data
#new_data['Tweet_clean'] = new_data['Tweet'].apply(clean_text)
X_new = vectorizer.transform(new_data['Tweet'])
X_new_selected = selector.transform(X_new)

# Predict the labels for the new data
y_new_pred = nb.predict(X_new_selected)
new_data['Predicted Label'] = y_new_pred

# Save the predictions to a new Excel file
new_data.to_excel("Adhani_clean_pred.xlsx", index=False)


In [39]:
new_data.head()

,Unnamed: 0.1,Unnamed: 0,Date,User,Tweet,Location,profile,likes,retweets,followerCount,Predicted Label
0,0,0,2023-03-12 13:27:11+00:00,linu_nair,ushanirmala hidenburg adani pyar mein baki bho...,NaN,indian,0,0,17,CAG
1,1,1,2023-03-12 13:16:43+00:00,linu_nair,wonder hidenburg cover may occupied adani,NaN,indian,0,0,17,CAG
2,2,2,2023-03-12 12:55:29+00:00,StockMarketTyms,hidenburg try best shake indianmarkets many ve...,NaN,proud year experience stock,0,0,21,NAG
3,3,3,2023-03-12 12:24:01+00:00,ShivayRudra,hidenburg busy make fake report adani country ...,new delhi india,NaN,0,0,62,NAG
4,5,5,2023-03-12 10:02:53+00:00,viprayami1,narendramodi pmoindia crash shut biggest bank ...,canada,buy kid meritocratic dev dharm jaati desh easi...,0,0,91,NAG


In [40]:
new_data['Predicted Label'].value_counts()

CAG    15681
NAG    10631
OAG     3352
Name: Predicted Label, dtype: int64

In [41]:
###End: 

# Predictions of Shinde Govt

In [49]:
import pandas as pd

# Load the Excel file with the new data
new_data = pd.read_excel("Shinde_clean.xlsx")

# Replace missing values with empty strings
new_data['Tweet'] = new_data['Tweet'].fillna('')


# Preprocess the new data
#new_data['Tweet_clean'] = new_data['Tweet'].apply(clean_text)
X_new = vectorizer.transform(new_data['Tweet'])
X_new_selected = selector.transform(X_new)

# Predict the labels for the new data
y_new_pred = nb.predict(X_new_selected)
new_data['Predicted Label'] = y_new_pred

# Save the predictions to a new Excel file
new_data.to_excel("Shinde_clean_pred.xlsx", index=False)


In [50]:
new_data['Predicted Label'].value_counts()

CAG    6598
NAG    2532
OAG    1341
Name: Predicted Label, dtype: int64

In [51]:
new_data.head()

,Unnamed: 0,Date,User,Tweet,Location,profile,likes,retweets,followerCount,Predicted Label
0,0,2023-03-26 20:31:33+00:00,MayurShinde4,anyone attack govt report btw demand jpc reque...,NaN,NaN,1,0,8,CAG
1,1,2023-03-26 18:24:51+00:00,VinayShindeBlr,enemywithin putin yanukovych jinping get modi ...,bharat union state,NaN,0,0,966,OAG
2,2,2023-03-26 15:09:07+00:00,pareshshinde38,blblvishwa ani stop financial base reservation...,NaN,NaN,0,0,4,CAG
3,3,2023-03-26 14:06:55+00:00,ShashankNair2,maharashtra shinde meet mns chief rajthackeray...,NaN,journalist news,2,2,73,CAG
4,4,2023-03-26 12:22:54+00:00,SRH_IS_LUB,shindeshahiinmaharashtra shinde bulldozer man ...,milkyway galaxy,NaN,1,21,197,CAG


# Prediction for Cricket

In [30]:
import pandas as pd

# Load the Excel file with the new data
new_data = pd.read_excel("Cricket_clean.xlsx")

# Replace missing values with empty strings
new_data['Tweet'] = new_data['Tweet'].fillna('')


# Preprocess the new data
#new_data['Tweet_clean'] = new_data['Tweet'].apply(clean_text)
X_new = vectorizer.transform(new_data['Tweet'])
X_new_selected = selector.transform(X_new)

# Predict the labels for the new data
y_new_pred = nb.predict(X_new_selected)
new_data['Predicted Label'] = y_new_pred

# Save the predictions to a new Excel file
new_data.to_excel("Cricket_clean_pred.xlsx", index=False)


In [31]:
new_data.head()

,Unnamed: 0,Date,User,Tweet,Location,profile,likes,retweets,followerCount,Predicted Label
0,0,2022-12-29 13:33:02+00:00,mkaif_786,jbsy bowl lineup ayi world cup pakistan final ...,cricket,pct pakistan cricket important student,3.0,0.0,2416.0,NAG
1,1,2022-12-28 13:04:06+00:00,Ammi42737578,pakistan newzland test safique babar wasim eng...,NaN,abðÿ,0.0,0.0,0.0,CAG
2,2,2022-12-27 03:25:59+00:00,rathoure_b,viratgodfather tab opposition tab india team j...,NaN,anime love,0.0,0.0,12.0,NAG
3,3,2022-12-25 14:41:22+00:00,SwaradevB,icc would like pakistan semifinal world cup fo...,rayagada india,nature lover twitter use person,0.0,0.0,23.0,NAG
4,5,2022-12-24 09:20:43+00:00,Drjunaid79,vibhubhola still pakistan play world cup semif...,peshawar pakistan,study save life vet doctor uvasian uvas,1.0,0.0,130.0,OAG


In [32]:
new_data['Predicted Label'].value_counts()

NAG    1744
OAG    1441
CAG     721
Name: Predicted Label, dtype: int64

# Shraddha Walker Prediction

In [30]:
import pandas as pd

# Load the Excel file with the new data
new_data = pd.read_excel("datasetSW.xlsx")

# Replace missing values with empty strings
new_data['Tweet'] = new_data['Tweet'].fillna('')


# Preprocess the new data
#new_data['Tweet_clean'] = new_data['Tweet'].apply(clean_text)
X_new = vectorizer.transform(new_data['Tweet'])
X_new_selected = selector.transform(X_new)

# Predict the labels for the new data
y_new_pred = nb.predict(X_new_selected)
new_data['Predicted Label'] = y_new_pred

# Save the predictions to a new Excel file
new_data.to_excel("datasetSW_Prediction.xlsx", index=False)


In [31]:
new_data['Predicted Label'].value_counts()

CAG    10581
OAG     5263
NAG     2463
Name: Predicted Label, dtype: int64